In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

#Reset notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

import math
import numpy as np
import scipy.constants as const

from pykern.pkcollections import PKDict

import rslaser.rspulse.pulse as rsp
import rslaser.rspulse.gauss_hermite as rsgh
import rslaser.utils.plot_fields as rspf
import rslaser.utils.constants as rsc

In [ ]:
# specify all top-level input parameters

# central values
a0 = 0.01            # 0.85e-9 * lambda [microns] * Sqrt(Intensity [W/cm^2])
lambda0 = 8.e-7      # wavelength [m]
tau_fwhm = 1.e-10    # FWHM pulse length [s]
w0 = 400.e-6         # waist size [m]
k0 = rsc.TWO_PI/lambda0    # central wavenumber [radians/m]
zR = 0.5*k0*w0*2           # Rayleigh range [m]
# z_waist = -15.*zR    # longitudinal waist position [m]
z_waist =  0.0       # longitudinal waist position [m]
z_center = 0.0       # longitudinal position of pulse center [m]

# deviations
d_lambda = 0.        # full wavelength chirp across laser pulse [m]
dw0x = 0.0           # deviation of horizontal waist from w0
dw0y = 0.0           # deviation of vertical waist from w0
dzwx = 0.0           # deviation of horizontal waist position from z_waist
dzwy = 0.0           # deviation of vertical waist position from z_waist
x_shift = 0.         # bulk horizontal shift of pulse position
y_shift = 0.         # bulk vertical shift of pulse position

# numerical values
num_slices = 3       # desired number of slices (i.e. SRW wavefronts) to represent the pulse
n_points = 101       # number of bins along each axis for contour plots

In [ ]:
# create a dictionary of all relevant physical and numerical quantities
data_in=PKDict()

data_in.a0 = a0
data_in.lambda0 = lambda0
data_in.tau_fwhm = tau_fwhm
data_in.w0 = w0
data_in.z_waist = z_waist
data_in.z_center = z_center
data_in.d_to_w = z_waist - z_center

data_in.d_lambda = d_lambda
data_in.dw0x = dw0x
data_in.dw0y = dw0y
data_in.dzwx = dzwx
data_in.dzwy = dzwy
data_in.x_shift = x_shift
data_in.y_shift = y_shift

data_in.nslice = num_slices

#create laser pulse
LP = rsp.LaserPulse(data_in)

In [ ]:
# specify the desired plots
plot_2d_zy_explicit = False
plot_1d_z_explicit = False
plot_1d_z_envelope = False
plot_1d_x_envelope = True

In [ ]:
# plot fields in a z-y plane that intersects the x=0 axis
# we are assuming linear polarization along the x-axis
if plot_2d_zy_explicit:
    with plt.style.context(('seaborn-poster')):
        fig = plt.figure()
        ax = fig.gca()
        rspf.plot_zy(LP.envelope, ax)

In [ ]:
# plot a lineout of Ex along z
# we are assuming linear polarization along the x-axis
# the time-dependent oscillations will be included

if plot_1d_z_explicit:
    z_max = 40. * LP.envelope.lambda0
    z_min = 0.

    # resolution
    z_res = 32
    del_z = lambda0 / z_res
    num_pts = int((z_max - z_min) / del_z)
    print('\n  num_pts = ', num_pts)

    z_vals = np.zeros(num_pts)
    for ind in range(num_pts):
        z_vals[ind] = z_min + ind*del_z
    
    x_val = 0.
    y_val = 0.
    t_val = 0.
    time_explicit = True

    with plt.style.context(('seaborn-poster')):
        fig = plt.figure()
        ax = fig.gca()
        rspf.plot_1d_z(z_vals, LP.envelope, ax, x_val, y_val, t_val, time_explicit)

    plt.tight_layout()

In [ ]:
# plot a lineout of Ex along z
# we are assuming linear polarization along the x-axis
# this time, we plot only the envelope

if plot_1d_z_envelope:
    z_max = 3. * LP.envelope.tau_fwhm * const.c
    z_min = -z_max

    # resolution
    z_res = 4
    del_z = LP.envelope.w0 / z_res
    num_pts = int((z_max - z_min) / del_z)
    # print('\n  num_pts = ', num_pts)

    z_vals = np.zeros(num_pts)
    for ind in range(num_pts):
        z_vals[ind] = z_min + ind*del_z
    
    x_val = 0.
    y_val = 0.
    t_val = 0.
    time_explicit = False

    with plt.style.context(('seaborn-poster')):
        fig = plt.figure()
        ax = fig.gca()
        rspf.plot_1d_z(z_vals, LP.envelope, ax, x_val, y_val, t_val, time_explicit)

    plt.tight_layout()

In [ ]:
# let's consider a diverging pulse
data_in.z_waist = -80.*LP.envelope.zR
data_in.d_to_w = z_waist - z_center

# print('data_in = ', data_in)

# instantiate a new laser pulse
LP2 = rsp.LaserPulse(data_in)

In [ ]:
# plot a lineout of Ex along x
# we are assuming linear polarization along the x-axis

if plot_1d_x_envelope:
    x_max = 300. * LP2.envelope.w0
    x_min = -x_max
    
    print(' x_max = ', x_max)
    print(' z_waist = ', LP2.envelope.z_waist)

    # resolution
    x_res = 0.01
    del_x = LP2.envelope.lambda0 / x_res
    num_pts = int((x_max - x_min) / del_x)
    print('\n  num_pts = ', num_pts)

    x_vals = np.zeros(num_pts)
    for ind in range(num_pts):
        x_vals[ind] = x_min + ind*del_x
    
    y_val = 0.
    z_val = LP2.envelope.L_fwhm

    with plt.style.context(('seaborn-poster')):
        fig = plt.figure()
        ax = fig.gca()
        rspf.plot_1d_x(x_vals, LP2.envelope, ax, y_val, z_val)

    plt.tight_layout()